Mount drive for Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Check dataset directory

In [ ]:
!ls /content/drive/MyDrive/fort/obj_train_data/images/train
!ls /content/drive/MyDrive/fort/obj_train_data/images/val
!ls /content/drive/MyDrive/fort/obj_train_data/images/test

!cat /content/drive/MyDrive/fort/obj_train_data/dataset.yaml

Clone YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

Install requirements

In [ ]:
!pip install -r requirements.txt

Train the model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 60 --data /content/drive/MyDrive/fort/obj_train_data/dataset.yaml --weights yolov5s.pt

Save model weights

In [ ]:
!cp runs/train/exp/weights/best.pt /content/drive/MyDrive/fort/knockdown_model_weights.pt

Load YOLO model

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/fort/knockdown_model_weights.pt')

Evaluate model

In [ ]:
!python val.py --weights /content/drive/MyDrive/fort/knockdown_model_weights.pt --data /content/drive/MyDrive/fort/obj_train_data/dataset.yaml --task test

In [ ]:
results_base_dir = '/content/yolov5/runs/val/exp'
os.listdir(results_base_dir)

In [ ]:
import os
import random
import cv2
from IPython.display import display, Image
import matplotlib.pyplot as plt
import torch

test_images_dir = '/content/drive/MyDrive/fort/fortnite_data'

test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.png') or img.endswith('.jpg')]

random_img_path = random.choice(test_images)
print(f"Selected image: {random_img_path}")

def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv1.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

img = cv2.imread(random_img_path)

img_resized = cv2.resize(img, (640, 640))

model_path = '/content/drive/MyDrive/fortnite_yolo/knockdown_model_weights.pt'
model = torch.hub.load('/content/yolov5', 'custom', path=model_path, source='local')

model.conf = 0.1

print("Running inference...")
results = model(img_resized)

print(results)
print(results.xyxy)

if results.xyxy[0].shape[0] > 0:
    confidences = results.xyxy[0][:, 4].cpu().numpy()
    print(f"Confidence scores: {confidences}")

    for *box, conf, cls in results.xyxy[0]:
        label = f'{model.names[int(cls)]} {conf:.2f}'
        plot_one_box(box, img_resized, label=label, color=(255, 0, 0), line_thickness=2)

    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

    temp_img_path = random_img_path.replace('.png', '_conf_display.png').replace('.jpg', '_conf_display.jpg')
    cv2.imwrite(temp_img_path, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))

    display(Image(filename=temp_img_path))
else:
    print("No detections found in the image.")

img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 10))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()


In [ ]:
import cv2
from matplotlib import pyplot as plt
import os
from IPython.display import display, Image


results_dir = '/content/yolov5/runs/val/exp'


result_images = ['val_batch0_pred.jpg', 'val_batch0_labels.jpg']

def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

for img_name in result_images:
    img_path = os.path.join(results_dir, img_name)
    if os.path.exists(img_path):
        print(f"Processing image: {img_path}")
        img = cv2.imread(img_path)
        results = model(img_path)

        for *box, conf, cls in results.xyxy[0]:
            label = f'{model.names[int(cls)]} {conf:.2f}'
            plot_one_box(box, img, label=label, color=(255, 0, 0), line_thickness=2)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        temp_img_path = img_path.replace('.jpg', '_conf_display.jpg')
        cv2.imwrite(temp_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        display(Image(filename=temp_img_path))
    else:
        print(f"Image not found: {img_path}")
